# 8-bit MIPS Design & Simulation

## Instruction Set

![Instructions](instructions.png)

## Assigned Instructions
| Instruction | Opcode |
|:-----------:|:------:|
|   **D** (subi)  |  0X00  |
|   **C** (sub)   |  0X01  |
|   **J** (srl)   |  0X02  |
|   **O** (bneq)  |  0X03  |
|    **G** (or)   |  0X04  |
|    **P** (j)    |  0X05  |
|   **A** (add)   |  0X06  |
|   **F** (andi)  |  0X07  |
|   **B** (addi)  |  0X08  |
|   **I** (sll)   |  0X09  |
|   **N** (beq)   |  0X0A  |
|   **H** (ori)   |  0X0B  |
|   **E** (and)   |  0X0C  |
|   **K** (nor)   |  0X0D  |
|    **L** (sw)   |  0X0E  |
|    **M** (lw)   |  0X0F  |

## Main Control

| Instruction | OpCode | ALUOp | RegDst | Jump | Branch | MemRead | MemtoReg | MemWrite | ALUSrc | RegWrite | HexCode |
|-------------|:------:|:-----:|:------:|:----:|:------:|:-------:|:--------:|:--------:|:------:|:--------:|:-------:|
| subi        |  0x00  |  001  |    0   |   0  |    0   |    0    |     0    |     0    |    1   |     1    |  0x103  |
| sub         |  0x01  |  001  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |  0x181  |
| srl         |  0x02  |  110  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |  0x681  |
| bneq        |  0x03  |  111  |    0   |   0  |    1   |    0    |     0    |     0    |    0   |     0    |  0x720  |
| or          |  0x04  |  011  |    1   |   0  |    0   |    0    |     0    |     0    |    1   |     1    |  0x383  |
| j           |  0x05  |  000  |    0   |   1  |    0   |    0    |     0    |     0    |    0   |     0    |   0x40  |
| add         |  0x06  |  000  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |   0x81  |
| andi        |  0x07  |  010  |    0   |   0  |    0   |    0    |     0    |     0    |    1   |     1    |  0x203  |
| addi        |  0x08  |  000  |    0   |   0  |    0   |    0    |     0    |     0    |    1   |     1    |   0x3   |
| sll         |  0x09  |  101  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |  0x581  |
| beq         |  0x0A  |  000  |    0   |   0  |    1   |    0    |     0    |     0    |    0   |     0    |   0x20  |
| ori         |  0x0B  |  011  |    0   |   0  |    0   |    0    |     0    |     0    |    1   |     1    |  0x303  |
| and         |  0x0C  |  010  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |  0x281  |
| nor         |  0x0D  |  100  |    1   |   0  |    0   |    0    |     0    |     0    |    0   |     1    |  0x481  |
| sw          |  0x0E  |  000  |    0   |   0  |    0   |    0    |     0    |     1    |    1   |     0    |   0x6   |
| lw          |  0x0F  |  000  |    0   |   0  |    0   |    1    |     1    |     0    |    1   |     1    |   0x1B  |

## MIPS Code Processor

In [21]:
# Defining Constants

OPCODE = {
    "subi": "0000", "sub": "0001", "srl": "0010", "bneq": "0011",
    "or": "0100", "j": "0101", "add": "0110", "andi": "0111",
    "addi": "1000", "sll": "1001", "beq": "1010", "ori": "1011",
    "and": "1100", "nor": "1101", "sw": "1110", "lw": "1111"
}

REGISTERS = {
    "$zero": "0000", "$t0": "0001", "$t1": "0010", "$t2": "0011",
    "$t3": "0100", "$t4": "0101", "$sp": "0110"
}

In [30]:
# Main Processing Code
def mips_to_machine_code(mips_code):
    mips_code_list = [code.strip() for code in mips_code.split("\n") if len(code.strip()) > 1]
    labels = {code[0:code.find(":")]: idx for idx, code in enumerate(mips_code_list) if ":" in code}
    machine_code = ""
    for current_line_no, current_line in enumerate(mips_code_list):
        operation = current_line.split(" ")[0].lower()
        if operation in ["add", "sub", "and", "or", "nor"]:
            temp_line = current_line.replace(" ", "")
            registers = temp_line[temp_line.find("$"):].split(",")
            if len(registers) != 3:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            temp_machine_code = OPCODE[operation] \
                                + REGISTERS[registers[1]] \
                                + REGISTERS[registers[2]] \
                                + REGISTERS[registers[0]] \
                                + "0000"  # 0 for ShAmt
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "\n")
        elif operation in ["sll", "srl"]:
            temp_line = current_line.replace(" ", "")
            registers = temp_line[temp_line.find("$"):].split(",")
            if len(registers) != 3:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            temp_machine_code = OPCODE[operation] \
                                + "0000" \
                                + REGISTERS[registers[1]] \
                                + REGISTERS[registers[0]]  # 0 for RS
            sh_amt = bin(int(registers[2], 10))[2:]
            temp_machine_code += (("0" * (4 - len(sh_amt))) + sh_amt)
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "\n")
        elif operation in ["addi", "subi", "andi", "ori"]:
            temp_line = current_line.replace(" ", "")
            registers = temp_line[temp_line.find("$"):].split(",")
            if len(registers) != 3:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            temp_machine_code = OPCODE[operation] \
                                + REGISTERS[registers[1]] \
                                + REGISTERS[registers[0]]
            imm_data = bin(int(registers[2], 10))[2:]
            temp_machine_code += (("0" * (8 - len(imm_data))) + imm_data)
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "\n")
        elif operation in ["beq", "bneq"]:
            temp_line = current_line.replace(" ", "")
            registers = temp_line[temp_line.find("$"):].split(",")
            if len(registers) != 3:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            temp_machine_code = OPCODE[operation] \
                                + REGISTERS[registers[0]] \
                                + REGISTERS[registers[1]]
            relative_label = bin((labels[registers[2]] - (current_line_no + 1)))[2:]
            temp_machine_code += (("0" * (8 - len(relative_label))) + relative_label)
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "\n")
        elif operation in ["sw", "lw"]:
            temp_line = current_line.replace(" ", "")
            registers = temp_line[temp_line.find("$"):].split(",")
            if len(registers) != 2:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            temp_machine_code = OPCODE[operation] \
                                + REGISTERS[registers[1][registers[1].find("(")+1:registers[1].find(")")]] \
                                + REGISTERS[registers[0]]
            imm_data = bin(int(registers[1][:registers[1].find("(")]))
            temp_machine_code += ("0" * (8 - len(imm_data)) + imm_data[2:])
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "\n")
        elif operation == "j":
            temp_machine_code = OPCODE["j"]
            temp_tokens = current_line.split(" ")
            if len(temp_tokens) != 2:
                print(f"Error: at line {current_line_no} : {current_line}")
                continue
            jump_address = bin(labels[temp_tokens[1]])[2:]
            temp_machine_code += (("0" * (8 - len(jump_address))) + jump_address)
            machine_code += (hex(int(temp_machine_code, 2))[2:] + "00\n")
        else:
            if ":" not in current_line:
                print(f"Error: at line {current_line_no}: {current_line}")
            else:
                machine_code += "40000\n"

    return machine_code

In [31]:
# FILE IO
INPUT_FILE = "mips_code.txt"
OUTPUT_FILE = "machine_code.txt"

in_file = open(INPUT_FILE, "r")
mips_code = in_file.read()
in_file.close()

out_file = open(OUTPUT_FILE, "w")
machine_code = mips_to_machine_code(mips_code)
out_file.write("v2.0 raw\n")
out_file.write(machine_code)
out_file.close()